In [2]:
import numpy as np
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Convolution2D, MaxPooling2D, Flatten
from keras.optimizers import Adam

### 1_載入數據

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [4]:
print('x_shape: ', x_train.shape)
print('y_shape: ', y_train.shape)

x_shape:  (60000, 28, 28)
y_shape:  (60000,)


### 2_數據集處理

In [7]:
# (60000, 28, 28) -> (60000, 28, 28, 1)  # 轉成四維格式
x_Train = x_train.reshape(-1, 28, 28, 1) / 255.0   # (data筆數, 長, 寬, 1(黑白圖片))
x_Test = x_test.reshape(-1, 28, 28, 1) / 255.0 

In [8]:
# 對label轉成one-hot
y_Train_onehot = np_utils.to_categorical(y_train, num_classes=10)
y_Test_onehot = np_utils.to_categorical(y_test, num_classes=10)

### 3-1_創建模型

In [10]:
# 定義順序模型
model = Sequential()

# 開始加層別
# 第一層卷積層(Convolution Layer)   # (28,28,1)
model.add(Convolution2D(
    input_shape=(28,28,1),
    filters=32,  # 32個filter
    kernel_size=5,  # filter的大小:5*5
    strides=1,  #步長
    padding='same',
    activation='relu'
))

# 第一層池化層(Pooling Layer) # (14,14,32)
model.add(MaxPooling2D(
    pool_size=2,
    strides=2,
    padding='same'
))

# 第二層卷積層   # (14,14,64)
model.add(Convolution2D(64, 5, strides=1, padding='same', activation='relu'))  

# 第二層池化層   # (7,7,64)
model.add(MaxPooling2D(2,2,'same'))

# 將第二層池化層的輸出扁平化為1維  # (7,7,64) -> (7*7*64)  為了當下一層FC的輸入
model.add(Flatten())

# 第一個全連接層(Fully Connected)
model.add(Dense(1024, activation='relu'))   # 輸入維度:7*7*64 -> 輸出維度:1024

# Dropout
model.add(Dropout(0.5))

# 第二個全連接層
model.add(Dense(10, activation='softmax'))   # 輸入維度:1024 -> 輸出維度:10(手寫數字0~9)


In [11]:
# 定義優化器，loss function, 並在訓練過程中計算準確率
adam = Adam(learning_rate=1e-4)

# 編譯
model.compile(
    optimizer = adam,
    loss = 'categorical_crossentropy',   # 此處只改loss function為交叉熵
    metrics = ['accuracy']
)

### 3-2 訓練模型

In [12]:
model.fit(x_Train, y_Train_onehot, batch_size=64, epochs=10)

Epoch 1/10
60000/60000 [==============================] - 138s 2ms/step - loss: 0.3468 - accuracy: 0.9000
Epoch 2/10
60000/60000 [==============================] - 137s 2ms/step - loss: 0.0966 - accuracy: 0.9707
Epoch 3/10
60000/60000 [==============================] - 138s 2ms/step - loss: 0.0648 - accuracy: 0.9807
Epoch 4/10
60000/60000 [==============================] - 144s 2ms/step - loss: 0.0512 - accuracy: 0.9843
Epoch 5/10
60000/60000 [==============================] - 145s 2ms/step - loss: 0.0431 - accuracy: 0.9867
Epoch 6/10
60000/60000 [==============================] - 140s 2ms/step - loss: 0.0360 - accuracy: 0.9888
Epoch 7/10
60000/60000 [==============================] - 132s 2ms/step - loss: 0.0309 - accuracy: 0.99041s - loss: 0.0
Epoch 8/10
60000/60000 [==============================] - 133s 2ms/step - loss: 0.0281 - accuracy: 0.9911
Epoch 9/10
60000/60000 [==============================] - 134s 2ms/step - loss: 0.0239 - accuracy: 0.9921
Epoch 10/10
60000/60000 [=======

### 3-3_評估模型

In [13]:
loss, accuracy = model.evaluate(x_Test, y_Test_onehot)

print('\ntest loss: ', loss)
print('accuracy: ', accuracy)

10000/10000 [==============================] - 4s 411us/step

test loss:  0.023133614263590424
accuracy:  0.9919999837875366


### 小結：
* 使用了卷積神經網絡模型後，準確率大大提升至99%。